In [58]:
import numpy as np
import pandas as pd
import ads
ads.config.token = "LjXPoovayDU3qk4SIi8l1JwAuhhIwmGUUWJTSHN2"




In [98]:
def adapt_key_string(reference, target_name, column):
    # print('      ')
    # print('REFERENCE: ')
    # print(reference)
    # print('      ')
    
    key_string = ''
    if type(reference) == str:
        if reference != 'This Work':
            try:
                bibcode = reference.split('/')[-2]
            except:
                print(reference)
            bibcode = bibcode.replace('A%26A', 'A&A')
            # print('bibcode: ', bibcode)
            search_result = ads.SearchQuery(bibcode=bibcode)
            for result in search_result:
                key_string = result.first_author.split(',')[0] + result.year
        else:
            key_string = 'This Work'

        if "ó" in key_string:
            key_string = key_string.replace(r'ó', 'o')
        if "ö" in key_string:
            key_string = key_string.replace(r'ö', 'o')
        if "Ö" in key_string:
            key_string = key_string.replace(r'Ö', 'O')
        if "ł" in key_string:
            key_string = key_string.replace(r'ł', 'l')
        if "ȩ" in key_string:
            key_string = key_string.replace(r'ȩ', 'e')
        if "ú" in key_string:
            key_string = key_string.replace(r'ú', 'u')
        if "ñ" in key_string:
            key_string = key_string.replace(r'ñ', 'n')
        if "á" in key_string:
            key_string = key_string.replace(r'á', 'a')
        if "Á" in key_string:
            key_string = key_string.replace(r'Á', 'A')
        if "é" in key_string:
            key_string = key_string.replace(r'é', 'e')
        if "ê" in key_string:
            key_string = key_string.replace(r'ê', 'e')
        if "É" in key_string:
            key_string = key_string.replace(r'É', 'E')
        if "í" in key_string:
            key_string = key_string.replace(r'í', 'i')
        if "ü" in key_string:
            key_string = key_string.replace(r'ü', 'u')

        if key_string == "Glebocki2005":
            key_string = "Glebocki2005b"
        if (key_string == "Bouvier1986"):
            if target_name == 'DN Tau':
                key_string = "Bouvier1986b"
            else:
                key_string = "Bouvier1986a"
        if (key_string == "Kraus2009") and (column == 'Teff (K)'):
            key_string = "Kraus2009b"
        if (key_string == "Maiz-Apellaniz2023") or (key_string == "Maiz Apellaniz2023"):
            key_string = "MaizApellaniz2023"
        if key_string == "Cruz-Saenz de Miera2023":
            key_string = "Cruz-SaenzdeMiera2023"
        if key_string == "Lopez-Martinez2015":
            key_string = "Lopez-Martinez2015a"
        if key_string == "Gaia Collaboration2022":
            key_string = "GaiaCollaboration2022"
        if key_string == "van der Marel2018":
            key_string = "vanderMarel2018"
        if key_string == "van der Marel2016":
            key_string = "vanderMarel2016a"

        if "van " in key_string:
            key_string = key_string.replace("van ", "van")


        if key_string != "This Work":
            key_string = r"\citet{" + key_string + r"}"

        if key_string == r"\citet{\citet{Percy2010}}":
            key_string = r"\citet{Percy2010}"

    return key_string


def adapt_parameters(raw_Parameter, raw_Parameter_err, mean_Parameter, mean_Parameter_err, list_index, column):

    raw_Parameter_i = raw_Parameter[list_index]
    raw_Parameter_err_i = raw_Parameter_err[list_index]

    # --------- ERR VALUES FOR RAW SOURCE VALUES --------- #
    if type(raw_Parameter_err_i) == str:
        if ',' in raw_Parameter_err_i:
            err_p = float(raw_Parameter_err_i.split(',')[0])
            err_m = float(raw_Parameter_err_i.split(',')[1])
            raw_Parameter_err_i = str(np.round(np.mean([err_p, err_m]),2))
    else:
        if np.isnan(raw_Parameter_err_i) == True:
            raw_Parameter_err_i = '-'
        else:
            if column == 'Teff (K)':
                raw_Parameter_err_i = str(int(raw_Parameter_err_i))
            else:
                raw_Parameter_err_i = str(raw_Parameter_err_i)
    

    # --------- QUOTED VALUES FOR RAW SOURCE VALUES --------- #
    if type(raw_Parameter_i) != str:
        if np.isnan(raw_Parameter_i) == True:
            raw_Parameter_i = '-'
        else:
            if column == 'Teff (K)':
                raw_Parameter_i = str(int(raw_Parameter_i))
            else:
                raw_Parameter_i = str(raw_Parameter_i)

    else:
        if '<' in raw_Parameter_i:
            raw_Parameter_i = raw_Parameter_i.replace('<', '$<$')
            raw_Parameter_err_i = '-'
            mean_Parameter_err = '-'


    # --------- ADOPTED MEAN VALUES --------- #
    if type(mean_Parameter) != str:
        if np.isnan(mean_Parameter) == True:
            mean_Parameter = '-'
        else:
            if column == 'Teff (K)':
                mean_Parameter = str(int(mean_Parameter))
            else:
                mean_Parameter = str(mean_Parameter)
    else:
        if (column == 'Teff (K)') and '.' in mean_Parameter:
            mean_Parameter = str(int(float(mean_Parameter)))
    
     # --------- ADOPTED MEAN ERR VALUES --------- #
    if type(mean_Parameter_err) != str:
        if np.isnan(mean_Parameter_err) == True:
            mean_Parameter_err = '-'
        else:
            if column == 'Teff (K)':
                mean_Parameter_err = str(int(mean_Parameter_err))
            else:
                mean_Parameter_err = str(mean_Parameter_err)
    else:
        if (column == 'Teff (K)') and '.' in mean_Parameter_err:
            mean_Parameter_err = str(int(float(mean_Parameter_err)))
            
        if mean_Parameter_err == 'nan':
            mean_Parameter_err = '-'


    return raw_Parameter_i, raw_Parameter_err_i, mean_Parameter, mean_Parameter_err


def initialize_reference_table_text(tabulated_property):
    if tabulated_property == 'vsini (km/s)':
        table_caption_text = "Adopted Projected Rotational Velocities and References To Literature Sources"
        table_label = "vsinis"
        table_colhead = r"\vsini"
        table_colhead_unit = r"(km s$^{-1}$)"
    elif tabulated_property == 'Prot (d)':
        table_caption_text = "Adopted Stellar Rotation Periods and References To Literature Sources"
        table_label = "Prots"
        table_colhead = r"\Prot"
        table_colhead_unit = "(d)"
    elif tabulated_property == 'Rstar (Rsun)':
        table_caption_text = "Adopted Stellar Radii and References To Literature Sources"
        table_label = "Rstars"
        table_colhead = r"\Rstar"
        table_colhead_unit = r"(\Rsun)"
    elif tabulated_property == 'i pp disk (deg)':
        table_caption_text = "Adopted Primordial Disk Inclinations and References To Literature Sources"
        table_label = "iDisks"
        table_colhead = r"\idisk"
        table_colhead_unit = r"($\degr$)"
    elif tabulated_property == 'SpType':
        table_caption_text = "Adopted Stellar Spectral Types and References To Literature Sources"
        table_label = "SpTypes"
        table_colhead = "Spectral"
        table_colhead_unit = "Type"
    elif tabulated_property == 'Teff (K)':
        table_caption_text = "Adopted Stellar Effective Temperatures and References To Literature Sources"
        table_label = "Teffs"
        table_colhead = r"\Teff"
        table_colhead_unit = "(K)"
    elif tabulated_property == 'Mstar (Msun)':
        table_caption_text = "Adopted Stellar Masses and References To Literature Sources"
        table_label = "Mstars"
        table_colhead = r"\Mstar"
        table_colhead_unit = r"(\Msun)"

    latex_table_text = r"\startlongtable"
    latex_table_text += "\n"
    latex_table_text += r"\begin{deluxetable*}{lll}"
    latex_table_text += "\n"
    latex_table_text += r"\renewcommand\arraystretch{0.9}"
    latex_table_text += "\n"
    latex_table_text += r"\tabletypesize{\small}"
    latex_table_text += "\n"
    latex_table_text += r"\setlength{\tabcolsep}{6pt}"
    latex_table_text += "\n"
    latex_table_text += r"\tablewidth{0pt}"
    latex_table_text += "\n"
    latex_table_text += r"\tablecolumns{3}"
    latex_table_text += "\n"
    latex_table_text += r"\tablecaption{" + table_caption_text + r"\label{tab:" + table_label + r"}}"
    latex_table_text += "\n"
    latex_table_text += r"\tablehead{"
    latex_table_text += "\n"
    latex_table_text += r"Object   &   " + table_colhead + r"   &   Reference \\"
    latex_table_text += "\n"
    latex_table_text += r"\vspace{-16pt}  \\"
    latex_table_text += "\n"
    latex_table_text += r"Name     &   " + table_colhead_unit + r"   &   "
    latex_table_text += "\n"
    latex_table_text += r"}"
    latex_table_text += "\n"
    latex_table_text += r"\startdata"
    latex_table_text += "\n"

    return latex_table_text





In [99]:
sheet_url = "https://docs.google.com/spreadsheets/d/1xuMXHVTw3_iwn3tgzBoIaXaMP81ULbpIURthZyvs7iw/edit#gid=0"
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

raw_df = pd.read_csv(url_1)
raw_df.columns


Index(['Target', 'True Name', 'RA', 'Dec', 'SpType', 'SpType Letter',
       'SpType Number', 'SpType Reference', 'i pp disk (deg)', 'i pp disk Unc',
       'i pp disk Reference', 'Continuum Disk Size (au)',
       'Continuum Disk Size Unc', 'Notes on Continuum Disk Size',
       'Rstar (Rsun)', 'Rstar Unc', 'Rstar Reference', 'Mstar (Msun)',
       'Mstar Unc', 'Mstar Reference', 'Prot (d)', 'Prot Unc',
       'Prot Reference', 'LC Source', 'vsini (km/s)', 'vsini Unc',
       'vsini Reference', 'Teff (K)', 'Teff Unc', 'Teff Reference',
       'Lstar (Lsun)', 'logLstar (Lsun)', 'Lstar Unc', 'Lstar Reference',
       'Age (Myr)', 'logAge', 'Age Unc', 'Age Reference', 'Notes',
       'Alternate Name', 'SED Class', 'Multiplicity', 'Multiplicity Reference',
       'Circumbinary Flag (1=yes)',
       'Tasks to ensure the sample is complete and accurate', 'Unnamed: 45'],
      dtype='object')

In [100]:
primordial_disks_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/'
primordial_disks_sample_directory = primordial_disks_directory+'Tables/Condensed_Sample_By_Date/'
test_figures_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/Test_Figures/'

HBM_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/HBM/Lauren_epop_analysis_2024-March-6/Final_HBM_Files/'

In [101]:
R_shift = int(1)
P_shift = int(1)
sampling_resolution = 'high'  # 'high'  # 'med'  # 'low'

Rshift_text = 'Rshift' + str(R_shift)
Pshift_text = 'Pshift' + str(P_shift)
resolution_text = sampling_resolution + 'Res'

date_str = '2024-Apr-17'

sample_filepath_stassun = primordial_disks_sample_directory + 'adopted_' + resolution_text + '_' + date_str + '.csv'

In [102]:
condensed_df_stassun = pd.read_csv(sample_filepath_stassun)

condensed_df_stassun['vsini Unc'].replace('ulimit', 0.0000, inplace=True)
condensed_df_stassun['vsini Unc'] = condensed_df_stassun['vsini Unc'].astype(float)
condensed_df_stassun = condensed_df_stassun.sort_values('Simbad Name')

exclude_objects = ['HP Tau', 'DL Tau']
for object_i in exclude_objects:
    condensed_df_stassun = condensed_df_stassun[condensed_df_stassun['Simbad Name'] != object_i]

# condensed_df_stassun.to_latex()

In [103]:

table_columns = {'vsini': {'parameter colhead': 'vsini (km/s)',
                           'err colhead': 'vsini Unc',
                           'reference colhead': 'vsini Reference',
                           },
                'Prot': {'parameter colhead': 'Prot (d)',
                           'err colhead': 'Prot Unc',
                           'reference colhead': 'Prot Reference',
                           },
                'Rstar': {'parameter colhead': 'Rstar (Rsun)',
                           'err colhead': 'Rstar Unc',
                           'reference colhead': 'Rstar Reference',
                           },
                'idisk': {'parameter colhead': 'i pp disk (deg)',
                           'err colhead': 'i pp disk Unc',
                           'reference colhead': 'i pp disk Reference',
                           },
                'Teff': {'parameter colhead': 'Teff (K)',
                           'err colhead': 'Teff Unc',
                           'reference colhead': 'Teff Reference',
                           },
                'Mstar': {'parameter colhead': 'Mstar (Msun)',
                           'err colhead': 'Mstar Unc',
                           'reference colhead': 'Mstar Reference',
                           },
                }

In [105]:

table_parameter = 'Mstar'

column_text = table_columns[table_parameter]['parameter colhead']
column_err_text = table_columns[table_parameter]['err colhead']
column_ref_text = table_columns[table_parameter]['reference colhead']

latex_table = initialize_reference_table_text(tabulated_property=column_text)


completed = []

for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
    target_name = condensed_df_stassun['Simbad Name'].values[target_i]
    if target_name not in completed:
        # if (target_name == 'CX Tau') or (target_name == 'AS 209'):

        mean_Parameter = str(round(condensed_df_stassun[column_text].values[target_i], 2))
        mean_Parameter_err = str(round(condensed_df_stassun[column_err_text].values[target_i], 2))

        temp_df = raw_df[raw_df['Target'] == target_name]
        temp_df = temp_df[pd.isnull(temp_df[column_text]) == False]
        raw_references = temp_df[column_ref_text].values
        raw_Parameter = temp_df[column_text].values
        raw_Parameter_err = temp_df[column_err_text].values

        # print(target_name)

        for ref_i in range(len(raw_Parameter)):

            if '2MASS' in target_name:
                target_name = target_name.split(' ')[-1]

            key_string = adapt_key_string(reference=raw_references[ref_i], target_name=target_name, column=column_text)
            
            raw_Parameter_i, raw_Parameter_err_i, mean_Parameter, mean_Parameter_err = adapt_parameters(raw_Parameter, raw_Parameter_err, mean_Parameter, mean_Parameter_err, list_index=ref_i, column=column_text)


            if ref_i == 0:
                if raw_Parameter_err_i != '-':
                    latex_table += target_name + r' & ' + raw_Parameter_i + r'$\pm$' + raw_Parameter_err_i + r' & ' + key_string + r' \\'
                else:
                    latex_table += target_name + r' & ' + raw_Parameter_i + r' & ' + key_string + r' \\'
            else:
                if raw_Parameter_err_i != '-':
                    latex_table += r' & ' + raw_Parameter_i + r'$\pm$' + raw_Parameter_err_i + r' & ' + key_string + r' \\'
                else:
                    latex_table += r' & ' + raw_Parameter_i + r' & ' + key_string + r' \\'

            latex_table += "\n"

        if mean_Parameter_err != '-':
            latex_table += r' & ' + r'\textbf{' + mean_Parameter+ r'$\pm$' + mean_Parameter_err + r'}' + r' & ' + r'\textbf{Adopted}' + r' \\'
        else:
            latex_table += r' & ' + r'\textbf{' + '$<$' + mean_Parameter + r'}' + r' & ' + r'\textbf{Adopted}' + r' \\'
        latex_table += "\n"
        latex_table += r'\hline'
        latex_table += "\n"
    completed.append(target_name)


    
latex_table += r"\enddata"
latex_table += "\n"
latex_table += r"\end{deluxetable*}"

print("\n\n")
print(latex_table)

APIResponseError: '{\n  "error": "Too many requests"\n}\n'

In [ ]:
def get_max_rows(df, colheads):
    maxrows = 0
    for colhead_i in range(len(colheads)):
        colrows = df[colheads[colhead_i]].values
        realrows = 0
        for colrow in range(len(colrows)):
            if type(colrows[colrow]) == str:
                realrows += 1
            elif np.isnan(colrows[colrow]) == False:
                realrows += 1

        if realrows > maxrows:
            maxrows = realrows
    return maxrows


In [ ]:
# =================================================================================================== #
# ============================    OTHER STELLAR PARAMETERS AND SOURCES     ========================== #
# =================================================================================================== #

latex_table = r"\startlongtable"
latex_table += "\n"
latex_table += r"\begin{rotatetable*}"
latex_table += "\n"
latex_table += r"\begin{deluxetable*}{lclclcl}"
latex_table += "\n"
latex_table += r"\renewcommand\arraystretch{0.9}"
latex_table += "\n"
latex_table += r"\tabletypesize{\small}"
latex_table += "\n"
latex_table += r"\setlength{\tabcolsep}{6pt}"
latex_table += "\n"
latex_table += r"\tablewidth{0pt}"
latex_table += "\n"
latex_table += r"\tablecolumns{7}"
latex_table += "\n"
latex_table += r"\tablecaption{Bep.\label{tab:}}"
latex_table += "\n"
latex_table += r"\tablehead{"
latex_table += "\n"
latex_table += r"\colhead{Object}   &   \colhead{Spectral}     &   \colhead{Reference}     &   \colhead{\Teff}     &   \colhead{Reference}      &   \colhead{\Mstar}      &   \colhead{Reference}  \\"
latex_table += "\n"
latex_table += r"\vspace{-16pt}  \\"
latex_table += "\n"
latex_table += r"\colhead{Name}     &   \colhead{Type}         &  \colhead{}                &   \colhead{(K)}      &   \colhead{}               &   \colhead{(\Msun)}      &   \colhead{ } "
latex_table += "\n"
latex_table += r"}"
latex_table += "\n"
latex_table += r"\startdata"
latex_table += "\n"



completed = []

column_text = ['SpType', 'Teff (K)', 'Mstar (Msun)']
column_err_text = ['SpType Unc', 'Teff Unc', 'Mstar Unc']
column_ref_text = ['SpType Reference', 'Teff Reference', 'Mstar Reference']

column_dict = {'SpType': [],
               'Teff (K)': [],
               'Mstar (Msun)': [],
               }

SpType_designations = {'3': 'G',
                       '4': 'K',
                       '5': 'M',
                       }
keys_list = list(SpType_designations.keys())
SpType_vals = [int(key) for key in keys_list]
               

for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):

    target_name = condensed_df_stassun['Simbad Name'].values[target_i]

    if '2MASS' in target_name:
        target_name = target_name.split(' ')[-1]

    if target_name not in completed:
    # if target_name == 'AA Tau':

        temp_df = raw_df[raw_df['Target'] == target_name]

        raw_rows = get_max_rows(df=temp_df, colheads=column_text)
        # print(raw_rows)

        mean_Parameters = ['']*len(column_text)
        mean_Parameter_errs = ['']*len(column_text)

        raw_Parameters_text = []
        raw_Parameter_errs_text = []

        for column_text_i in range(len(column_text)):

            raw_Parameters = ['']*raw_rows
            raw_Parameter_errs = ['']*raw_rows
            raw_references = ['']*raw_rows

            if column_text[column_text_i] == 'SpType':
                mean_Parameters[column_text_i] = condensed_df_stassun[column_text[column_text_i]].values[target_i]
                mean_Parameter_errs[column_text_i] = condensed_df_stassun[column_err_text[column_text_i]].values[target_i]
                mean_SpType_str = ''
                for SpType_val_i in range(len(SpType_vals)):
                    if int(np.floor(mean_Parameters[column_text_i])) == SpType_vals[SpType_val_i]:
                        mean_SpType_str += SpType_designations[str(SpType_vals[SpType_val_i])]
                        mean_SpType = 10*(mean_Parameters[column_text_i] - SpType_vals[SpType_val_i])
                        mean_SpType_str += str(round(mean_SpType, 1))
                        # mean_SpType_err_str += str(10*(mean_Parameters[column_text_i] - SpType_vals[SpType_val_i]))[0]
                
                # adopted_par_text = r'\textbf{' + mean_SpType_str  # + r'$\pm$' + mean_Parameter_errs[column_text_i] + r'}'
                adopted_par_text = r'\textbf{' + mean_SpType_str + r'}   &  \textbf{Adopted}'

            else:
                mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))

                if column_text[column_text_i] == 'Teff (K)':
                    mean_Parameters[column_text_i] = str(int(condensed_df_stassun[column_text[column_text_i]].values[target_i]))
                    mean_Parameter_errs[column_text_i] = str(int(condensed_df_stassun[column_err_text[column_text_i]].values[target_i]))
                else:
                    mean_Parameters[column_text_i] = str(round(condensed_df_stassun[column_text[column_text_i]].values[target_i], 2))
                    mean_Parameter_errs[column_text_i] = str(round(condensed_df_stassun[column_err_text[column_text_i]].values[target_i], 2))
                
                adopted_par_text = r'\textbf{' + mean_Parameters[column_text_i] + r'$\pm$' + mean_Parameter_errs[column_text_i] + r'}   &  \textbf{Adopted}'
            

            raw_Parameters_i = temp_df[column_text[column_text_i]].values

            if column_text[column_text_i] == 'SpType':
                raw_Parameter_errs_i = ['']*len(raw_Parameters_i)
            else:
                raw_Parameter_errs_i = temp_df[column_err_text[column_text_i]].values
            raw_references_i = temp_df[column_ref_text[column_text_i]].values

            for raw_i in range(raw_rows):
                raw_par_str = ''
                raw_par_err_str = ''
                raw_ref_str = ''

                if column_text[column_text_i] == 'SpType':
                    if type(raw_Parameters_i[raw_i]) == str:
                        raw_par_str = raw_Parameters_i[raw_i]

                    raw_ref_str = adapt_key_string(reference=raw_references_i[raw_i])

                else:
                    if np.isnan(raw_Parameters_i[raw_i]) == False:

                        if column_text[column_text_i] == 'Teff (K)':
                            raw_par_str = str(int(raw_Parameters_i[raw_i]))

                        raw_par_str = raw_Parameters_i[raw_i]

                        if type(raw_par_str) != str:
                            if column_text[column_text_i] == 'Teff (K)':
                                raw_par_str = str(int(raw_par_str))
                            else:
                                raw_par_str = str(raw_par_str)

                        if type(raw_Parameter_errs_i[raw_i]) == str:
                            if ',' in raw_Parameter_errs_i[raw_i]:
                                err_p = float(raw_Parameter_errs_i[raw_i].split(',')[0])
                                err_m = float(raw_Parameter_errs_i[raw_i].split(',')[1])
                                if column_text[column_text_i] == 'Teff (K)':
                                    raw_par_err_str = str(int(np.mean([err_p, err_m])))
                                else:
                                    raw_par_err_str = str(np.round(np.mean([err_p, err_m]),2))
                            else:
                                if column_text[column_text_i] == 'Teff (K)':
                                    raw_par_err_str = str(int(float(raw_Parameter_errs_i[raw_i])))
                                else:
                                    raw_par_err_str = str(raw_Parameter_errs_i[raw_i])
                        
                        elif type(raw_Parameter_errs_i[raw_i]) == float:
                            if np.isnan(raw_Parameter_errs_i[raw_i]) == True:
                                raw_par_err_str = ''

                        raw_ref_str = adapt_key_string(reference=raw_references_i[raw_i])


                if raw_par_err_str != '':
                    cell_text = raw_par_str + r'$\pm$' + raw_par_err_str + r'  &  ' + raw_ref_str
                else:
                    cell_text = raw_par_str + r'  &  ' + raw_ref_str


                raw_Parameters[raw_i] = cell_text

            
            column_dict[column_text[column_text_i]] = raw_Parameters
            column_dict[column_text[column_text_i]].append(adopted_par_text)


        for row_i in range(raw_rows+1):

            if row_i == 0:
                latex_table += target_name

            for column_text_i in range(len(column_text)):
                latex_table += r'  &  ' + column_dict[column_text[column_text_i]][row_i]

            latex_table += r' \\'

            latex_table += "\n"

        
                
        # # print(target_name)

        latex_table += r'\hline'
        latex_table += "\n"
        completed.append(target_name)

latex_table += r"\enddata"
latex_table += "\n"
latex_table += r"\end{deluxetable*}"
latex_table += "\n"
latex_table += r"\end{rotatetable*}"
latex_table += "\n"


print("\n\n")
print(latex_table)

In [ ]:
condensed_df_stassun.columns

In [ ]:
def round_sig(number, sigfigs=2):
    return round(number, sigfigs-int(np.floor(np.log10(abs(number))))-1)


inner_disks_url = "https://docs.google.com/spreadsheets/d/1Mmk4XNZFJxL4MTQ_asGtadOgtrVZlJzoo-lAr87_cAc/edit#gid=0"
inner_disks_url_1 = inner_disks_url.replace('/edit#gid=', '/export?format=csv&gid=')

inner_disks_df = pd.read_csv(inner_disks_url_1)
inner_disks_df.columns

In [ ]:
inner_disks_targets = [] 

for inner_disk_target_name in inner_disks_df['Target'].values:
        
    if '2MASS' in inner_disk_target_name:
            inner_disk_target_name = inner_disk_target_name.split(' ')[-1]

    if inner_disk_target_name not in inner_disks_targets:
        inner_disks_targets.append(inner_disk_target_name)

In [ ]:
# =================================================================================================== #
# ==========================+====    ADOPTED PARAMETERS FOR ANALYSIS     ============================ #
# =================================================================================================== #

show_evidence_for_inner_disk = False
if show_evidence_for_inner_disk == True:
    latex_table = r"\begin{deluxetable*}{lccccccc}[ht!]"
else:
    latex_table = r"\begin{deluxetable*}{lcccccc}[ht!]"
latex_table += "\n"
latex_table += r"\renewcommand\arraystretch{0.9}"
latex_table += "\n"
latex_table += r"\tabletypesize{\small}"
latex_table += "\n"
latex_table += r"\setlength{\tabcolsep}{6pt}"
latex_table += "\n"
latex_table += r"\tablewidth{0pt}"
latex_table += "\n"
if show_evidence_for_inner_disk == True:
    latex_table += r"\tablecolumns{8}"
else:
    latex_table += r"\tablecolumns{7}"
latex_table += "\n"
latex_table += r"\tablecaption{Adopted star and disk parameters used for primary analysis. Values of \vsini, \Prot, and \Rstar given below are the weighted mean of all known literature values compiled for each object. The full list of all compiled values of \vsini, \Prot, and \Rstar and references to their sources is provided in Table \ref{tab:vsinis}, Table \ref{tab:Prots}, and Table \ref{tab:Rstars}, respectively. Sources for \idisk are given in Table \ref{tab:iDisk}. Values under \istar and \Deltai are the MAP values and 1\sig highest density intervals determined from the associated posterior distributions. \label{tab:Adopted_Star_Disk_Analysis_Parameters}}"
latex_table += "\n"
latex_table += r"\tablehead{"
latex_table += "\n"
if show_evidence_for_inner_disk == True:
    latex_table += r"\colhead{Object Name}  &  \colhead{\vsini}         &  \colhead{\Prot}  &  \colhead{\Rstar}   &  \colhead{\idisk}   &  \colhead{\istar}   &  \colhead{\Deltai}   &  \colhead{Evidence of}  \\"
    latex_table += "\n"
    latex_table += r"\vspace{-16pt}  \\"
    latex_table += "\n"
    latex_table += r"\colhead{}    &  \colhead{(km s$^{-1}$)}  &  \colhead{(d)}    &  \colhead{(\Rsun)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}   &  \colhead{MID?}  "
else:
    latex_table += r"\colhead{Object Name}  &  \colhead{\vsini}         &  \colhead{\Prot}  &  \colhead{\Rstar}   &  \colhead{\idisk}   &  \colhead{\istar}   &  \colhead{\Deltai}   \\"
    latex_table += "\n"
    latex_table += r"\vspace{-16pt}  \\"
    latex_table += "\n"
    latex_table += r"\colhead{}    &  \colhead{(km s$^{-1}$)}  &  \colhead{(d)}    &  \colhead{(\Rsun)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  &  \colhead{($\degr$)}  "
latex_table += "\n"
latex_table += r"}"
latex_table += "\n"
latex_table += r"\startdata"
latex_table += "\n"



completed = []

column_list = ['vsini (km/s)', 'Prot (d)', 'Rstar (Rsun)', 'i pp disk (deg)', 'i star (deg)', 'disk diff', 'inner disk?']
column_err_p_list = ['vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star +Unc', 'disk diff +Unc', 'inner disk?']
column_err_m_list = ['vsini Unc', 'Prot Unc', 'Rstar Unc', 'i pp disk Unc', 'i star -Unc', 'disk diff -Unc', 'inner disk?']

for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
    target_name = condensed_df_stassun['Simbad Name'].values[target_i]

    if '2MASS' in target_name:
        target_name = target_name.split(' ')[-1]

    latex_table += target_name

    for col_i in range(len(column_list)):

        column_text = column_list[col_i]
        column_err_p_text = column_err_p_list[col_i]
        column_err_m_text = column_err_m_list[col_i]

        if column_text != 'inner disk?':

            mean_Parameter = round(condensed_df_stassun[column_text].values[target_i], 2)
            mean_Parameter_err_p = round(condensed_df_stassun[column_err_p_text].values[target_i], 2)
            mean_Parameter_err_m = round(condensed_df_stassun[column_err_m_text].values[target_i], 2)

            # print('target_name', 'raw_Parameter_i', 'raw_Parameter_err_i', 'key_string')
            # print(target_name, raw_Parameter_i, raw_Parameter_err_i, key_string)

            # if column_text != 'i star (deg)':
            #     latex_table += r' & ' + str(round_sig(number=mean_Parameter, sigfigs=2)) + r'$\pm$' + str(round_sig(number=mean_Parameter_err_p, sigfigs=2)) 
            # else:
            #     latex_table += r' & ' + str(round_sig(number=mean_Parameter, sigfigs=2)) + r'$^{+' + str(round_sig(number=mean_Parameter_err_p, sigfigs=2)) + r'}_{-' + str(round_sig(number=mean_Parameter_err_m, sigfigs=2)) + r'}'

            if (column_text != 'i star (deg)') and (column_text != 'disk diff'):
                latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$\pm$' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p)))
            else:
                latex_table += r' & ' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter))) + r'$^{+' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_p))) + r'}_{-' + "{:.2g}".format(float("{:.2g}".format(mean_Parameter_err_m))) + r'}$'
        else:
            if show_evidence_for_inner_disk == True:
                if target_name in inner_disks_targets:
                    latex_table += r' & ' + "Y"
                else:
                    latex_table += r' & ' + "-"


    latex_table += r'  \\'
    latex_table += "\n"
    completed.append(target_name)


    
latex_table += r"\enddata"
latex_table += "\n"
latex_table += r"\end{deluxetable*}"

print("\n\n")
print(latex_table)

In [ ]:
# =================================================================================================== #
# =============================    GENERAL STELLAR SAMPLE PARAMETERS     ============================ #
# =================================================================================================== #


latex_table = r"\begin{deluxetable}{lccc}[ht!]"
latex_table += "\n"
latex_table += r"\renewcommand\arraystretch{0.9}"
latex_table += "\n"
latex_table += r"\tabletypesize{\small}"
latex_table += "\n"
latex_table += r"\setlength{\tabcolsep}{6pt}"
latex_table += "\n"
latex_table += r"\tablewidth{0pt}"
latex_table += "\n"
latex_table += r"\tablecolumns{4}"
latex_table += "\n"
latex_table += r"\tablecaption{Other adopted stellar properties. Values reported here are the weighted mean of reported literature values. The full list of compiled values and their sources are given in Table \ref{tab:Compiled_Other_Stellar_Properties}.\label{tab:Other_Stellar_Properties}}"
latex_table += "\n"
latex_table += r"\tablehead{"
latex_table += "\n"
latex_table += r"\colhead{Object Name}  &  \colhead{Spectral}         &  \colhead{\Teff}  &  \colhead{\Mstar}  \\"
latex_table += "\n"
latex_table += r"\vspace{-16pt}  \\"
latex_table += "\n"
latex_table += r"\colhead{}    &  \colhead{Type}  &  \colhead{(K)}  &  \colhead{(\Msun)}   "
latex_table += "\n"
latex_table += r"}"
latex_table += "\n"
latex_table += r"\startdata"
latex_table += "\n"



completed = []

column_list = ['SpType', 'Teff (K)', 'Mstar (Msun)']
column_err_p_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc']
column_err_m_list = ['SpType Unc', 'Teff Unc', 'Mstar Unc']

for target_i in range(len(condensed_df_stassun['Simbad Name'].values)):
    target_name = condensed_df_stassun['Simbad Name'].values[target_i]

    if '2MASS' in target_name:
        target_name = target_name.split(' ')[-1]

    latex_table += target_name

    for col_i in range(len(column_list)):

        column_text = column_list[col_i]
        mean_Parameter = condensed_df_stassun[column_text].values[target_i]

        column_err_p_text = column_err_p_list[col_i]
        column_err_m_text = column_err_m_list[col_i]

        mean_Parameter_err_p = condensed_df_stassun[column_err_p_text].values[target_i]
        mean_Parameter_err_m = condensed_df_stassun[column_err_m_text].values[target_i]

        if column_text != 'SpType':
            
            if column_text != 'Teff (K)':
                latex_table += r' & ' + "{:.2f}".format(mean_Parameter) + r'$\pm$' + "{:.2f}".format(mean_Parameter_err_p)
            else:
                latex_table += r' & ' + "{:d}".format(int(mean_Parameter)) + r'$\pm$' + "{:d}".format(int(mean_Parameter_err_p))
        
        else:
            SpType_designations = {'3': 'G',
                                   '4': 'K',
                                   '5': 'M',
                                   }
            keys_list = list(SpType_designations.keys())
            SpType_vals = [int(key) for key in keys_list]
            SpType_text = ''
            SpType_err_text = ''
            for SpType_val_i in range(len(SpType_vals)):
                if int(np.floor(mean_Parameter)) == SpType_vals[SpType_val_i]:
                    SpType_text += SpType_designations[str(SpType_vals[SpType_val_i])]
                    SpType_text += str(10*(mean_Parameter - SpType_vals[SpType_val_i]))[0]
                    SpType_err_text += str(10*(mean_Parameter_err_m - SpType_vals[SpType_val_i]))[0]
               
            latex_table += r' & ' + SpType_text #  + r'$\pm$' + SpType_err_text



    latex_table += r'  \\'
    latex_table += "\n"
    completed.append(target_name)


    
latex_table += r"\enddata"
latex_table += "\n"
latex_table += r"\end{deluxetable}"

print("\n\n")
print(latex_table)